In [376]:
#### DEPRECATED, source of TravelingMusician_main ####

from fileinput import close
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import folium
from itertools import combinations
from IPython.display import HTML

#### Make Data ####

#open city files
city_names_text = open('../OIE project/Data/USCA312_labels.txt', 'r')
city_distances_text = open('../OIE project/Data/USCA312_distances.txt', 'r')


#put lines of city_names_Text in a list
city_names_list = [line.strip() for line in city_names_text]
#cut out the filler text at the beginning of the txt file
city_names_list = city_names_list[15:]

#put lines of city_names_Text in a list
city_dist_list = [line.strip() for line in city_distances_text]
#cut out the filler text at the beginning of the txt file
city_dist_list = city_dist_list[7:]
#split the individual numbers into their own elements, but they're split into sublists by line
city_dist_list = [line.split() for line in city_dist_list]
#merge all the sublists
city_dist_list = [element #return element, append to list
                    for sublist in city_dist_list #for each sublist
                        for element in sublist] #for each element in each sublist
#split the main list back up into chunks for each city
city_dist_list = [city_dist_list[city_index*len(city_names_list)
                                 :
                                 (city_index+1)*len(city_names_list)] #return a range that is a block of 312 cities
                        for city_index in range(len(city_names_list))] #for each city in city_names_list

#Create the dataframe
city_dist_df = pd.DataFrame(city_dist_list, #data is the distances list
                            index=city_names_list, #row and column indices are the city list
                            columns=city_names_list)

#Gurobi tutorial works with combinations instead so mild conversion
def distance(city1, city2):
    return int(city_dist_df[city1][city2])

#create a dict of all possible edges between cities
city_dist_dict = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(city_names_list, 2)}

#ok python why are you making me do this at the end of every block
print(len(list(city_dist_df.columns)))
city_names_list = list(city_dist_df.columns)
print(len(city_names_list))

312
312


In [377]:
#### Model Creation ###

## Coefficients and data ##

#Coach bus drivers can only drive 10 hours per day legally
#Rule of thumb in the industry is that 600 miles is the cap for a legal drive, see source below
# https://mcatransportation.com/legal-driving-hours/#:~:text=The%20basic%20rule%20of%20thumb,also%20make%20the%20trip%20safe.
#Driving range can be altered for future testing
driver_range = 600

#touring list is the required cities
touring_route_list = ['Los Angeles, CA', 'New York, NY', 'Dallas, TX']
#create the dict of touring cities
touring_route_dict = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(touring_route_list, 2)}

#Eliminate all items where the distance is above the max range
#This step implicitly implies a constraint to not drive further than allowed for 

#put the items to be deleted in a list
bad_keys = []
for key in city_dist_dict:
    if city_dist_dict[key] > driver_range:
        bad_keys.append(key)


#delete keys in list from dict
for key in bad_keys:
    del city_dist_dict[key]


#All non-required cities go in this list
other_cities_list = city_names_list
for city in touring_route_list:
    other_cities_list.remove(city)
    
other_cities_dist_dict = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(other_cities_list, 2)}

#I have no clue why but there are sections where the city_names_list loses half of its values
#so I stuck this in every block to preserve its structure
city_names_list = list(city_dist_df.columns)

In [378]:
## Base model ##

m = gp.Model()

#add binary variables for every possible edge
vars = m.addVars(city_dist_dict.keys(), obj=city_dist_dict, vtype=GRB.BINARY, name='x')
#add additional binary variables for every city edge
other_city_binary_vars = m.addVars(other_cities_list, vtype=GRB.BINARY, name='support')

#while the problem is symmetrical, no edge will be traversed twice 
#so we can treat the edges as assymetrical

#ok python why are you making me do this at the end of every block
city_names_list = list(city_dist_df.columns)

In [379]:
## City constraints ## 

#Any mandatory cities must be entered and exited
mandatory_cities_in = m.addConstrs(vars.sum(city, '*') == 1 for city in touring_route_list)
mandatory_cities_out = m.addConstrs(vars.sum('*', city) == 1 for city in touring_route_list)

#All the other cities must either be entered and exited OR left completely alone
other_cities_sumtotal = m.addConstrs((2*other_city_binary_vars[city]) + vars.sum(city, '*') == 2 for city in other_cities_list)

#ok python why are you making me do this at the end of every block
city_names_list = list(city_dist_df.columns)

In [380]:
## Subtour constraints ##

#Lazy subtour constraints
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        # if routes are shorter than a mandatory tour
        if len(tour) < len(touring_route_list):
            # add subtour elimination constr. for every pair of cities in subtour
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# Given a tuplelist of edges, find the shortest subtour
def subtour(edges):
    unvisited = city_names_list[:]
    cycle = city_names_list[:] # Dummy - guaranteed to be replaced
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) < len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

#ok python why are you making me do this at the end of every block
city_names_list = list(city_dist_df.columns)

In [381]:
#### Solving the model ####

#solve
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

#ok python why are you making me do this at the end of every block
city_names_list = list(city_dist_df.columns)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 315 rows, 12871 columns and 12647 nonzeros
Model fingerprint: 0x79c700a4
Variable types: 0 continuous, 12871 integer (12871 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 21, time in user-callback 0.00 sec


In [382]:
#### sort through bullshit ####

all_edges = vars.select('*','*')

unsorted_route_cities = []
unsorted_route_tuples = []

for edge in all_edges:
    if edge.x > 0.5:
        edge_name = edge.VarName
        edge_name = edge_name[edge_name.find('[')+1 : edge_name.find(']')]
        split_point = edge_name.find(',')+4
        start_node = edge_name[0:split_point]
        end_node = edge_name[split_point+1:]
        unsorted_route_tuples.append((start_node, end_node))
        unsorted_route_cities.append(start_node)
        unsorted_route_cities.append(end_node)

#helper function to sort the cities
#sorts the tuples by where the last city was
#(city 1, city2) -> (city2, city3)
def sort_cities(edge_list):
    #get first edge as start of list
    sorted_list = [edge_list[0]]
    #get the destination of eedge 1
    end_node = edge_list[0][0]

    #skip the first tuple because it's already processed
    for edge in edge_list[1:]:
        #if the city you are leaving from is the same as the destination from the last node it's next in line
        if edge[0] == end_node:
            #put the next in line edge in the sorted list
            sorted_list.append(edge)
            #search for the next in line for the current pair
            end_node = edge[1]

    return sorted_list


#print(unsorted_route_cities)
print(unsorted_route_tuples)
print(
    [distance(tup[0], tup[1]) for tup in unsorted_route_tuples]
)
#print(sort_cities(unsorted_route_tuples))

#ok python why are you making me do this at the end of every block
city_names_list = list(city_dist_df.columns)


AttributeError: Unable to retrieve attribute 'x'

In [ ]:
#### Generate Map ####

## Note: The points are all a little bit off of the actual cities because all the lat/long values are rounded
#data for all the lats and longs of the cities
city_coords_df = pd.read_csv('../OIE project/Data/USCA312_coords.csv')

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in unsorted_route_cities:
  city_index = city_names_list.index(city)   
  points.append( #Append the lat and long as a tuple
    (city_coords_df.iloc[city_index]['Latitude'],
    city_coords_df.iloc[city_index]['Longitude'])
  )

folium.PolyLine(points).add_to(map)
for point in points:
    folium.Marker(point).add_to(map)

map